In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://cdn.downtoearth.org.in/library/large/2020-03-01/0.01792700_1583044755_coronavirus-illustration-carousel.jpg")

# COVID-19 Global Outlook: modelling, prediction and sentiment analysis 

In the context of the global COVID-19 pandemic, we follow the suggestions from Kaggle's competitions in order to provide useful insights about the virus' spread. Starting from a global exploratory analysis, then we focus on virus' modelling and prediction for the countries with the largest number of confirmed cases. For modelling, we implement SIR Model with some extensions and, for prediction, logistic and Gompertz model. At the end, we choose the best model based on R2 score, check the predictions' numbers about confirmed and fatalities for the next time interval and display some results from NLTK Sentiment analysis. 

Data: [COVID19 Global Forecasting](https://www.kaggle.com/c/covid19-global-forecasting-week-4)
Consulted kernels: 

**TABLE OF CONTENTS**

1. [Exploratory data analysis (EDA)](#section1)

    1.1. [Worldwide Trend](#section11)
    
    1.2. [Country-Wise growth](#section12)
    
    1.3. [Zoom up to](#section13)
    
      1.3.1. [US](#section131)
      
      1.3.2. [Europe](#section132)
      
      1.3.3. [Asia](#section133)
      
    
2. [Modelling](#section2)

    2.1. [SIR model](#section21)
    
    2.2. [SIR-Model with Lockdown](#section22)
    
      2.2.1. [Fitting SIR with Lockdown to data](#section131)
      
    2.3. [SIR with time-dependent R_0 and CFR](#section21)
    
      2.3.1. [Fitting extended SIR to data](#section131)
    
    
3. [Prediction](#section3)

    3.1. [Logistic model](#section31)
    
    3.2. [Gompertz model](#section32)
    
    
4. [NLTK Sentiment Analysis](#section4)

    4.1. [Sentiment polarity](#section41)
  
    
5. [Conclusions](#section5)


In [ ]:
import os
import numpy as np
import pandas as pd
import scipy as sp

# --- plotly ---
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio
pio.templates.default = "plotly_white"

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
submission_example = pd.read_csv("../input/covid19-global-forecasting-week-4/submission.csv")
test = pd.read_csv("../input/covid19-global-forecasting-week-4/test.csv")
train = pd.read_csv("../input/covid19-global-forecasting-week-4/train.csv")

In [ ]:
train.rename({'Country_Region': 'country', 'Province_State': 'province', 'Id': 'id', 'Date': 'date', 'ConfirmedCases': 'confirmed', 'Fatalities': 'fatalities'}, axis=1, inplace=True)
test.rename({'Country_Region': 'country', 'Province_State': 'province', 'Id': 'id', 'Date': 'date', 'ConfirmedCases': 'confirmed', 'Fatalities': 'fatalities'}, axis=1, inplace=True)
train['country_province'] = train['country'].fillna('') + '/' + train['province'].fillna('')
test['country_province'] = test['country'].fillna('') + '/' + test['province'].fillna('')

# **1. Exploratory data analysis (EDA)** <a id="section1"></a>

### **1.1. Worldwide Trend** <a id="section11"></a>

In [ ]:
ww_df = train.groupby('date')[['confirmed', 'fatalities']].sum().reset_index()
ww_df['new_case'] = ww_df['confirmed'] - ww_df['confirmed'].shift(1)
ww_df.tail()

In [ ]:
ww_melt_df = pd.melt(ww_df, id_vars=['date'], value_vars=['confirmed', 'fatalities', 'new_case'])

In [ ]:
fig = px.line(ww_melt_df, x="date", y="value", color='variable', 
              title="Worldwide Confirmed/Death Cases Over Time")
fig.show()

In [ ]:
fig = px.line(ww_melt_df, x="date", y="value", color='variable',
              title="Worldwide Confirmed/Death Cases Over Time (Log scale)",
             log_y=True)
fig.show()

In [ ]:
ww_df['mortality'] = ww_df['fatalities'] / ww_df['confirmed']

fig = px.line(ww_df, x="date", y="mortality", 
              title="Worldwide Mortality Rate Over Time")
fig.show()

### **1.2. Country-Wise growth** <a id="section12"></a>

In [ ]:
country_df = train.groupby(['date', 'country'])[['confirmed', 'fatalities']].sum().reset_index()
target_date = country_df['date'].max()

print('Date: ', target_date)
for i in [1, 10, 100, 1000, 10000]:
    n_countries = len(country_df.query('(date == @target_date) & confirmed > @i'))
    print(f'{n_countries} countries have more than {i} confirmed cases')

In [ ]:
top_country_df = country_df.query('(date == @target_date) & (confirmed > 1000)').sort_values('confirmed', ascending=False)
top_country_df = top_country_df.iloc[0:7]
top_country_melt_df = pd.melt(top_country_df, id_vars='country', value_vars=['confirmed', 'fatalities'])

In [ ]:
fig = px.bar(top_country_melt_df.iloc[::-1],
             x='value', y='country', color='variable', barmode='group',
             title=f'Confirmed Cases/Deaths on {target_date}', text='value', height=1500, orientation='h')
fig.show()

In [ ]:
top10_countries = top_country_df.sort_values('confirmed', ascending=False).iloc[:10]['country'].unique()
top10_countries_df = country_df[country_df['country'].isin(top10_countries)]
fig = px.line(top10_countries_df,
              x='date', y='confirmed', color='country',
              title=f'Confirmed Cases for top 10 country as of {target_date}')
fig.show()

In [ ]:
top10_countries = top_country_df.sort_values('fatalities', ascending=False).iloc[:10]['country'].unique()
top10_countries_df = country_df[country_df['country'].isin(top10_countries)]
fig = px.line(top10_countries_df,
              x='date', y='fatalities', color='country',
              title=f'Fatalities for top 10 country as of {target_date}')
fig.show()

In [ ]:
top_country_df = country_df.query('(date == @target_date) & (confirmed > 100)')
top_country_df['mortality_rate'] = top_country_df['fatalities'] / top_country_df['confirmed']
top_country_df = top_country_df.sort_values('mortality_rate', ascending=False)

fig = px.bar(top_country_df[:15].iloc[::-1],
             x='mortality_rate', y='country',
             title=f'Mortality rate HIGH: top 15 countries on {target_date}', text='mortality_rate', height=800, orientation='h')
fig.show()

In [ ]:
fig = px.bar(top_country_df[-15:],
             x='mortality_rate', y='country',
             title=f'Mortality rate LOW: top 15 countries on {target_date}', text='mortality_rate', height=800, orientation='h')
fig.show()

In [ ]:
fig = px.scatter_geo(country_df, locations="country", locationmode='country names', 
                     color="confirmed", size='confirmed', hover_name="country", 
                     hover_data=['confirmed', 'fatalities'],
                     range_color= [0, country_df['confirmed'].max()], 
                     projection="natural earth", animation_frame="date", 
                     title='COVID-19: Confirmed cases spread Over Time', color_continuous_scale="portland")
# fig.update(layout_coloraxis_showscale=False)
fig.show()

In [ ]:
fig = px.scatter_geo(country_df, locations="country", locationmode='country names', 
                     color="fatalities", size='fatalities', hover_name="country", 
                     hover_data=['confirmed', 'fatalities'],
                     range_color= [0, country_df['confirmed'].max()], 
                     projection="natural earth", animation_frame="date", 
                     title='COVID-19: Fatalities growth Over Time', color_continuous_scale="portland")
fig.show()

## **1.3. Zoom up to** <a id="section13"></a>

### **1.3.1. US** <a id="section131"></a>

In [ ]:
usa_state_code_df = pd.read_csv("../input/usa-state-code/usa_states2.csv")

In [ ]:
# Prepare data frame only for US. 

train_us = train.query('country == "US"')
train_us['mortality_rate'] = train_us['fatalities'] / train_us['confirmed']

# Convert province column to its 2-char code name,
state_name_to_code = dict(zip(usa_state_code_df['state_name'], usa_state_code_df['state_code']))
train_us['province_code'] = train_us['province'].map(state_name_to_code)

# Only show latest days.
train_us_latest = train_us.query('date == @target_date')

In [ ]:
fig = px.choropleth(train_us_latest, locations='province_code', locationmode="USA-states",
                    color='confirmed', scope="usa", hover_data=['province', 'fatalities', 'mortality_rate'],
                    title=f'Confirmed cases in US on {target_date}')
fig.show()

In [ ]:
fig = px.choropleth(train_us_latest, locations='province_code', locationmode="USA-states",
                    color='mortality_rate', scope="usa", hover_data=['province', 'fatalities', 'mortality_rate'],
                    title=f'Mortality rate in US on {target_date}')
fig.show()

### **1.3.2. Europe** <a id="section132"></a>

In [ ]:
europe_country_list =list([
    'Austria','Belgium','Bulgaria','Croatia','Cyprus','Czechia','Denmark','Estonia','Finland','France','Germany','Greece','Hungary','Ireland',
    'Italy', 'Latvia','Luxembourg','Lithuania','Malta','Norway','Netherlands','Poland','Portugal','Romania','Slovakia','Slovenia',
    'Spain', 'Sweden', 'United Kingdom', 'Iceland', 'Russia', 'Switzerland', 'Serbia', 'Ukraine', 'Belarus',
    'Albania', 'Bosnia and Herzegovina', 'Kosovo', 'Moldova', 'Montenegro', 'North Macedonia'])

country_df['date'] = pd.to_datetime(country_df['date'])
train_europe = country_df[country_df['country'].isin(europe_country_list)]
#train_europe['date_str'] = pd.to_datetime(train_europe['date'])
train_europe_latest = train_europe.query('date == @target_date')

In [ ]:
fig = px.choropleth(train_europe_latest, locations="country", 
                    locationmode='country names', color="confirmed", 
                    hover_name="country", range_color=[1, 100000], 
                    color_continuous_scale='portland', 
                    title=f'European Countries with Confirmed Cases as of {target_date}', scope='europe', height=500)
fig.show()

In [ ]:
europe_country_list =list([
    'France','Germany',
    'Italy', 
    'Spain','United Kingdom','Belgium','Netherlands',"Austria","Portugal","Norway"])

country_df['date'] = pd.to_datetime(country_df['date'])
train_europe = country_df[country_df['country'].isin(europe_country_list)]
train_europe_march = train_europe.query('date > "2020-03-01"')
fig = px.line(train_europe_march,
              x='date', y='confirmed', color='country',
              title=f'Confirmed cases by country in Europe, as of {target_date}')
fig.show()

In [ ]:
fig = px.line(train_europe_march,
              x='date', y='fatalities', color='country',
              title=f'Fatalities by country in Europe, as of {target_date}')
fig.show()

In [ ]:
train_europe_march['prev_confirmed'] = train_europe_march.groupby('country')['confirmed'].shift(1)
train_europe_march['new_case'] = train_europe_march['confirmed'] - train_europe_march['prev_confirmed']
fig = px.line(train_europe_march,
              x='date', y='new_case', color='country',
              title=f'DAILY NEW Confirmed cases by country in Europe')
fig.show()

### **1.3.3. Asia** <a id="section133"></a>

In [ ]:
country_latest = country_df.query('date == @target_date')

fig = px.choropleth(country_latest, locations="country", 
                    locationmode='country names', color="confirmed", 
                    hover_name="country", range_color=[1, 50000], 
                    color_continuous_scale='portland', 
                    title=f'Asian Countries with Confirmed Cases as of {target_date}', scope='asia', height=800)
fig.show()

In [ ]:
china_df = train.query('country == "China"')
china_df['prev_confirmed'] = china_df.groupby('country')['confirmed'].shift(1)
china_df['new_case'] = china_df['confirmed'] - china_df['prev_confirmed']
china_df.loc[china_df['new_case'] < 0, 'new_case'] = 0.
fig = px.line(china_df,
              x='date', y='new_case', color='province',
              title=f'DAILY NEW Confirmed cases in China by province')
fig.show()

# **2. Modelling** <a id="section2"></a>

## **2.1. SIR Model** <a id="section21"></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import mpld3
mpld3.enable_notebook()

We have seen some general behavior of the virus in agregated data, for the country where the coronavirus was originated and for four other interesting countries. There's a lot of information to be extracted from this data; for example, we haven't analyzed the effects of long/lat of countries. However, since our main purpose is to develop a predective model in order to understand the key factors that impact the COVID-19 transmission, I'll move on to one of the most famous epidemiologic models: SIR. 

SIR is a simple model that considers a population that belongs to one of the following states:
1. **Susceptible (S)**. The individual hasn't contracted the disease, but she can be infected due to transmisison from infected people
2. **Infected (I)**. This person has contracted the disease
3. **Recovered/Deceased (R)**. The disease may lead to one of two destinies: either the person survives, hence developing inmunity to the disease, or the person is deceased. 

<img src="https://www.lewuathe.com/assets/img/posts/2020-03-11-covid-19-dynamics-with-sir-model/sir.png" width="500px">
Image by Kai Sasaki from [lewuathe.com](https://www.lewuathe.com/covid-19-dynamics-with-sir-model.html)

There are many versions of this model, considering birth and death (SIRD with demography), with intermediate states, etc. However, since we are in the early stages of the COVID-19 expansion and our interest is focused in the short term, we will consider that people develops immunity (in the long term, immunity may be lost and the COVID-19 may come back within a certain seasonality like the common flu) and there is no transition from recovered to the remaining two states. With this, the differential equations that govern the system are:

$$ {dS \over dt} = - {\beta S I \over N} $$

$$ {dI \over dt} = {\beta S I \over N} - \gamma I$$

$$ {dR \over dt} = \gamma I$$

Where $\beta$ is the contagion rate of the pathogen and $\gamma$ is the recovery rate.

In [ ]:
submission_example = pd.read_csv("../input/covid19-global-forecasting-week-4/submission.csv")
test = pd.read_csv("../input/covid19-global-forecasting-week-4/test.csv")
train = pd.read_csv("../input/covid19-global-forecasting-week-4/train.csv")

train["Country_Region"] = [country_name.replace("'","") for country_name in train["Country_Region"]]
train = train.groupby(['Country_Region', 'Date'], as_index=False)
train = train.aggregate(np.sum)

In [ ]:
from scipy.integrate import odeint

# The SIR model differential equations.
def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

We now want to add *cumulative* Deaths $X$ to the model: $X(t) = \textit{number of deaths from day 0 to day t}$ for $t\geq 14$, else $0$. 

Recursively, the number of cumulative deaths on day $t$ is equal to the number of cumulative deaths on day $t-1$ (that's $=X(t-1)$) plus the number of newly infected 13 days prior multiplied with the case fatality rate $\alpha$ (alpha) (I chose 13 days as that is reported as the average time from infection until death in [this study](https://wwwnc.cdc.gov/eid/article/26/6/20-0320_article)).

Now, the number of newly infected 13 days prior (that's the people who can die on day $t$) is equal to the number of infected 14 days prior multiplied with the expected amount of people an infected person infects per day (that's $\beta$). So the number of newly infected 13 days prior is $\beta \cdot I(t-14)$.

Putting it all together: $X(t) = X(t-1) + \alpha \cdot \beta \cdot I(t-14)$.

This is equal to the closed form formula $X(t) = \alpha \cdot \beta \cdot \displaystyle \sum_{i=0}^{t-14} I(i)$


In [ ]:
def SIR_model(N, D, R_0, CaseFatalityRate, max_days):
    '''
    N: total population
    D, R_0, CaseFatalityRate: see texts above
    '''
    I0, R0 = 1, 0  # Initial number of infected and recovered individuals (1 infected, 0 recovered) [this R0 has nothing to do with the basic reproduction number R0]
    S0 = N - I0 - R0 # Initial number of susceptible (everyone else)

    gamma = 1.0 / D  # see texts above
    beta = R_0 * gamma  # see texts above
    alpha = CaseFatalityRate

    t = np.linspace(0, max_days, max_days) # Grid of time points (in days)

    # Initial conditions vector
    y0 = S0, I0, R0
    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv, y0, t, args=(N, beta, gamma))
    S, I, R = ret.T

    # Adding deaths (see text above)
    X = np.zeros(max_days)
    for day in range(13, max_days):
        X[day] = sum(I[:day-13])
    X = alpha * beta * X


    # Plot the data on three separate curves for S(t), I(t) and R(t)
    f, ax = plt.subplots(1,1,figsize=(10,4))
    ax.plot(t, S, 'b', alpha=0.7, linewidth=2, label='Susceptible')
    ax.plot(t, I, 'y', alpha=0.7, linewidth=2, label='Infected')
    ax.plot(t, X, 'r', alpha=0.7, linewidth=2, label='Dead')
    ax.plot(t, R, 'g', alpha=0.7, linewidth=2, label='Recovered')

    ax.set_xlabel('Time (days)')
    ax.title.set_text('SIR-Model. Total Population: ' + str(N) + ", Days Infectious: " + str(D) + ", R_0: " + str(R_0) + ", CFR: " + str(CaseFatalityRate*100) + "%")
    # ax.set_ylabel('Number (1000s)')
    # ax.set_ylim(0,1.2)
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.grid(b=True, which='major', c='w', lw=2, ls='-')
    legend = ax.legend()
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
        ax.spines[spine].set_visible(False)
    plt.show();

# **2.2. SIR-Model with Lockdown** <a id="section22"></a>

We now want to find suitable parameters (Days infectious, R_0, CFR) for the SIR model

As I said before, the number of confirmed cases is likely far off from the real number (as not the whole population is getting tested) and thus is not very useful to fit our data to a SIR-Model.

So, we'll mainly use the number of deceased from the dataset to find parameters for the SIR model. What's important to note is that many countries implemented a *lockdown* that greatly reduces the basic reproduction number R_0; thus, we first tweak the model to allow for a second R_0_2 to come into effect on day L (for lockdown).

In [ ]:
def SIR_model_with_lockdown(N, D, R_0, CaseFatalityRate, max_days, L, R_0_2):
    '''
    N: total population
    D, R_0, CaseFatalityRate, ...: see texts above
    '''
    # BEFORE LOCKDOWN (same code as first model)
    I0, R0 = 1, 0  # Initial number of infected and recovered individuals (1 infected, 0 recovered) [this R0 has nothing to do with the basic reproduction number R0]
    S0 = N - I0 - R0 # Initial number of susceptible (everyone else)

    gamma = 1.0 / D  # see texts above
    beta = R_0 * gamma  # see texts above
    alpha = CaseFatalityRate

    t = np.linspace(0, L, L)  # Grid of time points (in days)
    
    # Initial conditions vector
    y0 = S0, I0, R0
    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv, y0, t, args=(N, beta, gamma))
    S, I, R = ret.T
    
    
    # AFTER LOCKDOWN
    I0_2, R0_2, S0_2 = I[-1], R[-1], S[-1]  # beginning of lockdown -> starting Infected/Susceptible/Recovered numbers are the numbers at the end of no-lockdown period

    gamma = 1.0 / D  # same after lockdown
    beta_2 = R_0_2 * gamma
    alpha = CaseFatalityRate  # same after lockdown

    t_2 = np.linspace(0, max_days - L + 1, max_days - L + 1)
    
    # Initial conditions vector
    y0_2 = S0_2, I0_2, R0_2
    # Integrate the SIR equations over the time grid, t.
    ret_2 = odeint(deriv, y0_2, t_2, args=(N, beta_2, gamma))
    S_2, I_2, R_2 = ret_2.T

    
    # COMBINING PERIODS
    S_full = np.concatenate((S, S_2[1:]))
    I_full = np.concatenate((I, I_2[1:]))
    R_full = np.concatenate((R, R_2[1:]))
    t_full = np.linspace(0, max_days, max_days)
    
    # Adding deaths
    X = np.zeros(max_days)
    for day in range(13, max_days):
        for valid_day in range(day-13):
            if valid_day < L:
                X[day] += alpha * beta * I_full[valid_day]
            else:
                X[day] += alpha * beta_2 * I_full[valid_day]

    

    # Plot the data on three separate curves for S(t), I(t) and R(t)
    f, ax = plt.subplots(1,1,figsize=(10,4))
    ax.plot(t_full, S_full, 'b', alpha=0.7, linewidth=2, label='Susceptible')
    ax.plot(t_full, I_full, 'y', alpha=0.7, linewidth=2, label='Infected')
    ax.plot(t_full, X, 'r', alpha=0.7, linewidth=2, label='Dead')
    ax.plot(t_full, R_full, 'g', alpha=0.7, linewidth=2, label='Recovered')

    ax.set_xlabel('Time (days)')
    ax.title.set_text('SIR-Model with Lockdown. Total Population: ' + str(N) + 
                      ", Days Infectious: " + str(D) + ", R_0: " + str(R_0) + 
                      ", CFR: " + str(CaseFatalityRate*100) + " R_0_2: " + str(R_0_2) + 
                      ", L: " + str(L) + " days")
    # ax.set_ylabel('Number (1000s)')
    # ax.set_ylim(0,1.2)
    plt.text(L,N/20,'Lockdown')
    plt.plot(L, 0, marker='o', markersize=6, color="red")
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.grid(b=True, which='major', c='w', lw=2, ls='-')
    legend = ax.legend()
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
        ax.spines[spine].set_visible(False)
    plt.show();

# **2.3 Fitting SIR with Lockdown to data** <a id="section23"></a>

We now try to fit the SIR-Model's Dead Curve to real data by tweaking the variables. Some of them are constant:
* max_days is set to len(train.groupby("Date").sum().index) so that we can compare against all available data
* N is fixed for each country, that's just the total population
* L is fixed for each country (the date it went into lockdown)
* D is set to vary from 5 to 20 (according to this study, it takes on avg. 5 days to show symptoms, at most 14; according to this source (German), people are infectious up to 5 days after onset of symptoms).
* CFR set to vary from  0.1%−10% 
* R_0 and R_0_2 are set to vary from 0.1 to 3.5

In [ ]:
def SIR_model_with_lockdown_deaths(x, N, D, R_0, CaseFatalityRate, max_days, L, R_0_2):
    # BEFORE LOCKDOWN (same code as first model)
    I0, R0 = 1, 0  # Initial number of infected and recovered individuals (1 infected, 0 recovered) [this R0 has nothing to do with the basic reproduction number R0]
    S0 = N - I0 - R0 # Initial number of susceptible (everyone else)

    gamma = 1.0 / D  # see texts above
    beta = R_0 * gamma  # see texts above
    alpha = CaseFatalityRate

    t = np.linspace(0, L, L)  # Grid of time points (in days)
    
    # Initial conditions vector
    y0 = S0, I0, R0
    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv, y0, t, args=(N, beta, gamma))
    S, I, R = ret.T
    
    
    # AFTER LOCKDOWN
    I0_2, R0_2, S0_2 = I[-1], R[-1], S[-1]  # beginning of lockdown -> starting Infected/Susceptible/Recovered numbers are the numbers at the end of no-lockdown period

    gamma = 1.0 / D  # same after lockdown
    beta_2 = R_0_2 * gamma
    alpha = CaseFatalityRate  # same after lockdown

    t_2 = np.linspace(0, max_days - L + 1, max_days - L + 1)
    
    # Initial conditions vector
    y0_2 = S0_2, I0_2, R0_2
    # Integrate the SIR equations over the time grid, t.
    ret_2 = odeint(deriv, y0_2, t_2, args=(N, beta_2, gamma))
    S_2, I_2, R_2 = ret_2.T

    
    # COMBINING PERIODS
    S_full = np.concatenate((S, S_2[1:]))
    I_full = np.concatenate((I, I_2[1:]))
    R_full = np.concatenate((R, R_2[1:]))
    t_full = np.linspace(0, max_days, max_days)
    
    # Adding deaths
    X = np.zeros(max_days)
    for day in range(13, max_days):
        for valid_day in range(day-13):
            if valid_day < L:
                X[day] += alpha * beta * I_full[valid_day]
            else:
                X[day] += alpha * beta_2 * I_full[valid_day]
    return X[x]

The (hidden as it's almost the same as before) code above defines a function with signature
SIR_model_with_lockdown_deaths(x, N, D, R_0, CaseFatalityRate, max_days, L, R_0_2)
that takes as input the same variables as before and an x and returns the number of fatalities on day x. This function will be used to find suited parameters D, CFR, R_0 and R_0_2 for the model.

In [ ]:
pip install lmfit

In [ ]:
from lmfit import Model

# Load countries data file (from https://www.kaggle.com/saga21/covid-global-forecast-sir-model-ml-regressions)
world_population = pd.read_csv("../input/population-by-country-2020/population_by_country_2020.csv")

# Select desired columns and rename some of them
world_population = world_population[['Country (or dependency)', 'Population (2020)', 'Density (P/Km²)', 'Land Area (Km²)', 'Med. Age', 'Urban Pop %']]
world_population.columns = ['Country (or dependency)', 'Population (2020)', 'Density', 'Land Area', 'Med Age', 'Urban Pop']

# Replace United States by US
world_population.loc[world_population['Country (or dependency)']=='United States', 'Country (or dependency)'] = 'US'

# Remove the % character from Urban Pop values
world_population['Urban Pop'] = world_population['Urban Pop'].str.rstrip('%')

# Replace Urban Pop and Med Age "N.A" by their respective modes, then transform to int
world_population.loc[world_population['Urban Pop']=='N.A.', 'Urban Pop'] = int(world_population.loc[world_population['Urban Pop']!='N.A.', 'Urban Pop'].mode()[0])
world_population['Urban Pop'] = world_population['Urban Pop'].astype('int16')
world_population.loc[world_population['Med Age']=='N.A.', 'Med Age'] = int(world_population.loc[world_population['Med Age']!='N.A.', 'Med Age'].mode()[0])
world_population['Med Age'] = world_population['Med Age'].astype('int16')

world_population.head(20)

We now define 
1. `fit_SIR`: this function takes a country name, lockdown data (and opt. region name) and first gathers the data (fatalities progression, population, etc.) and then fits the `SIR_model_with_lockdown_deaths`-function from above with fixed N (population), max_days (however many dates are supplied), L (lockdown date) and varying D, R_0, R_0_2, CFR. The function returns the lmfit-module's result object and the country name. The result object contains all we want to know about the curve fitting.
2. `fitted_plot`: this function takes a lmfit-result-object and country name and plots the fitted SIR-model against the real curve.

In [ ]:
lockdown_dates = {"Italy": "2020-03-10", "Spain": "2020-03-15", "US": "2020-03-20", "Germany": "2020-03-23", "France": "2020-03-17", "United Kingdom":"2020-03-23", "China":"2020-01-23", "Iran":"2020-03-25"}

def fit_SIR(country_name, lockdown_date=None):
    """
    y_data: the fatalities data of one country/region (array)
    population: total population of country
    lockdown_date: format YYYY-MM-DD
    """
    if lockdown_date is None:
        lockdown_date = lockdown_dates[country_name]
        
    y_data = train[(train["Country_Region"] == country_name)].Fatalities.values
    max_days = len(train.groupby("Date").sum().index) # constant for all countries

    # country specific values
    N = world_population.loc[world_population['Country (or dependency)'] == country_name]["Population (2020)"].values[0]
    L = train.groupby("Date").sum().index.tolist().index(lockdown_date)  # index of the lockdown date

    # x_data is just [0, 1, ..., max_days] array
    x_data = np.linspace(0, max_days - 1, max_days, dtype=int)
    
    # curve fitting from here
    mod = Model(SIR_model_with_lockdown_deaths)

    # initial values and bounds
    mod.set_param_hint('N', value=N)
    mod.set_param_hint('max_days', value=max_days)
    mod.set_param_hint('L', value=L)
    mod.set_param_hint('D', value=10, min=4, max=25)
    mod.set_param_hint('CaseFatalityRate', value=0.01, min=0.0001, max=0.1)
    mod.set_param_hint('R_0', value=2.0, min=0.1, max=5.0)
    mod.set_param_hint('R_0_2', value=2.0, min=0.1, max=5.0)

    params = mod.make_params()

    # fixing constant parameters
    params['N'].vary = False
    params['max_days'].vary = False
    params['L'].vary = False

    result = mod.fit(y_data, params, x=x_data, method="least_squares")
    
    return result, country_name

def fitted_plot(result, country_name, region_name=None):
    y_data = train[(train["Country_Region"] == country_name)].Fatalities.values
    max_days = len(train.groupby("Date").sum().index) # constant for all countries
    x_data = np.linspace(0, max_days - 1, max_days, dtype=int)
    x_ticks = train[train["Country_Region"] == "Germany"].Date.values  # same for all countries
    
    plt.figure(figsize=(10,5))
    
    real_data, = plt.plot(x_data, y_data, 'bo', label="real data")
    SIR_fit = plt.plot(x_data, result.best_fit, 'r-', label="SIR model")
    
    plt.xlabel("Day")
    plt.xticks(x_data[::10], x_ticks[::10])
    plt.ylabel("Fatalities")
    plt.title("Real Data vs SIR-Model in " + country_name)
    plt.legend(numpoints=1, loc=2, frameon=None)
    plt.show()

In [ ]:
result, _ = fit_SIR("Italy")
print(result.fit_report())
fitted_plot(result, "Italy")

In [ ]:
result, _ = fit_SIR("Spain")
print(result.fit_report())
fitted_plot(result, "Spain")

In [ ]:
result, _ = fit_SIR("Germany")
print(result.fit_report())
fitted_plot(result, "Germany")

In [ ]:
result, _ = fit_SIR("France")
print(result.fit_report())
fitted_plot(result, "France")

In [ ]:
result, _ = fit_SIR("US")
print(result.fit_report())
fitted_plot(result, "US")

In [ ]:
result, _ = fit_SIR("United Kingdom")
print(result.fit_report())
fitted_plot(result, "United Kingdom")

In [ ]:
result, _ = fit_SIR("China")
print(result.fit_report())
fitted_plot(result, "China")

In [ ]:
result, _ = fit_SIR("Iran")
print(result.fit_report())
fitted_plot(result, "Iran")

# **2.4 SIR with time-dependent R_0 and CFR** <a id="section24"></a>

While the prior models are able to capture some of the aspects of the virus quite well, it's not that hard to fit the curves to the outbreak period as they all look quite similar. To make better predictions, we now treat R_0 and CFR as functions. For example, there is no determined "Lockdown" date anymore at which R_0 jumps to a different value; it can now change continuously. Also, the CFR was until now treated as constant, however, with more people infected, treatment becomes less available and the case fatality rate increases. Now, CFR is treated as a function of the ratio $\frac{I(t)}{N}$ (the fraction of infected of the total population):

$CFR(t) = s \cdot \frac{I(t)}{N} + \alpha_{OPT}$, with $s$ being some arbitrary but fixed scaling factor and $\alpha_{OPT}$ being the CFR with optimal treatment available.

$R_{0}$ will be fitted to one of several different possible distributions we'll look at.

In [ ]:
def extended_deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta(t) * S * I / N
    dIdt = beta(t) * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

In [ ]:
def extended_SIR(N, D, max_days, CFR_OPT, CFR_scaling_factor, R_0, **R_0_kwargs):
    '''
    R_0: callable
    '''
    I0, R0 = 1, 0  # Initial number of infected and recovered individuals (1 infected, 0 recovered) [this R0 has nothing to do with the basic reproduction number R0]
    S0 = N - I0 - R0 # Initial number of susceptible (everyone else)

    gamma = 1.0 / D  # see texts above

    def beta(t):
        return R_0(t, **R_0_kwargs) * gamma

    t = np.linspace(0, max_days, max_days)  # Grid of time points (in days)
    
    # Initial conditions vector
    y0 = S0, I0, R0
    # Integrate the SIR equations over the time grid, t.
    ret = odeint(extended_deriv, y0, t, args=(N, beta, gamma))
    S, I, R = ret.T

    def CFR(t):
        return CFR_OPT + CFR_scaling_factor * (I[t] / N)

    # Adding deaths
    X = np.zeros(max_days)
    for day in range(13, max_days):
        for valid_day in range(day-13):
            X[day] += CFR(valid_day) * beta(valid_day) * I[valid_day]

    return t, S, I, R, X, [R_0(t, **R_0_kwargs) for t in range(max_days)], N, [CFR(t) for t in range(max_days)]

In [ ]:
def plot_extended_SIR(t, S, I, R, X, R_0, N, CFR):
    # Plot the data on three separate curves for S(t), I(t) and R(t)
    f, ax = plt.subplots(1,1,figsize=(10,4))
    ax.plot(t, S, 'b', alpha=0.7, linewidth=2, label='Susceptible')
    ax.plot(t, I, 'y', alpha=0.7, linewidth=2, label='Infected')
    ax.plot(t, X, 'r', alpha=0.7, linewidth=2, label='Dead')
    ax.plot(t, R, 'g', alpha=0.7, linewidth=2, label='Recovered')

    ax.set_xlabel('Time (days)')
    ax.title.set_text('SIR-Model with varying R_0 and CFR')
    # ax.set_ylabel('Number (1000s)')
    # ax.set_ylim(0,1.2)
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.grid(b=True, which='major', c='w', lw=2, ls='-')
    legend = ax.legend()
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
        ax.spines[spine].set_visible(False)
    plt.show();
    
    
    # plt.figure(figsize=(10,4))
    
    f = plt.figure(figsize=(10,4))
    
    # sp1
    ax1 = f.add_subplot(121)
    ax1.plot(t, R_0, 'b--', alpha=0.7, linewidth=2, label='R_0')
    
    ax1.set_xlabel('Time (days)')
    ax1.title.set_text('R_0 over time')
    # ax.set_ylabel('Number (1000s)')
    # ax.set_ylim(0,1.2)
    ax1.yaxis.set_tick_params(length=0)
    ax1.xaxis.set_tick_params(length=0)
    ax1.grid(b=True, which='major', c='w', lw=2, ls='-')
    legend = ax1.legend()
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
        ax.spines[spine].set_visible(False)

    # sp2
    ax2 = f.add_subplot(122)
    ax2.plot(t, CFR, 'r--', alpha=0.7, linewidth=2, label='CFR')
    
    ax2.set_xlabel('Time (days)')
    ax2.title.set_text('CFR over time')
    # ax.set_ylabel('Number (1000s)')
    # ax.set_ylim(0,1.2)
    ax2.yaxis.set_tick_params(length=0)
    ax2.xaxis.set_tick_params(length=0)
    ax2.grid(b=True, which='major', c='w', lw=2, ls='-')
    legend = ax2.legend()
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
        ax.spines[spine].set_visible(False)

    plt.show();

In [ ]:
N = 1_000
D = 4
max_days = 100

I0, R0 = 1, 0
S0 = N - I0 - R0
s = CFR_scaling_factor = 0.1
CFR_OPT = 0.02  # noone in hospital -> only 2% die

def new_R0(t, a, b, c):
    return a / (1 + (t/c)**b)


plot_extended_SIR(*extended_SIR(N, D, max_days, CFR_OPT, CFR_scaling_factor, new_R0, a=3.0, b=1.5, c=50))

## **2.4.1. Fitting extended SIR to data** <a id="section241"></a>

In [ ]:
def fit_extended_SIR(country_name, R_0_function, region_name=None, fit_method="least_squares", **R_0_kwargs):

    y_data = train[(train["Country_Region"] == country_name)].Fatalities.values
    max_days = len(train.groupby("Date").sum().index) # constant for all countries
    # country specific values
    N = world_population.loc[world_population['Country (or dependency)'] == country_name]["Population (2020)"].values[0]

    # x_data is just [0, 1, ..., max_days] array
    x_data = np.linspace(0, max_days - 1, max_days, dtype=int)

    # curve fitting from here
    def extended_SIR_deaths(x, N, D, max_days, CFR_OPT, CFR_scaling_factor, **R_0_kwargs):
        t_, S_, I_, R_, X, R_0_, N_, CFR_ = extended_SIR(N, D, max_days, CFR_OPT, CFR_scaling_factor, R_0=R_0_function, **R_0_kwargs)
        return X[x]

    mod = Model(extended_SIR_deaths)

    # initial values and bounds
    mod.set_param_hint('N', value=N, vary=False)
    mod.set_param_hint('max_days', value=max_days, vary=False)

    mod.set_param_hint('D', value=10, min=4, max=25)
    mod.set_param_hint('CFR_OPT', value=0.01, min=0.0001, max=0.1)
    mod.set_param_hint('CFR_scaling_factor', value=0.1, min=0.0001, max=1.0)
    if R_0_kwargs:
        for arg in R_0_kwargs:
            mod.set_param_hint(arg, value=R_0_kwargs[arg])

    params = mod.make_params()
    # print(params)
    result = mod.fit(y_data, params, method=fit_method, x=x_data)
    
    # fetch some result parameters
    CFR_OPT = result.params["CFR_OPT"].value
    CFR_scaling_factor = result.params["CFR_scaling_factor"].value
    R_0_result_params = {}
    for val in R_0_kwargs:
        R_0_result_params[val] = result.params[val].value

    
    # return result, country_name
    return result, country_name, N, D, max_days, CFR_OPT, CFR_scaling_factor, R_0_function, R_0_result_params

def fitted_plot_extended(result, country_name, region_name=None):
    y_data = train[(train["Country_Region"] == country_name)].Fatalities.values
    max_days = len(train.groupby("Date").sum().index) # constant for all countries
    x_data = np.linspace(0, max_days - 1, max_days, dtype=int)
    x_ticks = train[train["Country_Region"] == "Germany"].Date.values  # same for all countries
    
    plt.figure(figsize=(10,5))
    
    real_data, = plt.plot(x_data, y_data, 'bo', label="real data")
    SIR_fit = plt.plot(x_data, result.best_fit, 'r-', label="SIR model")
    
    plt.xlabel("Day")
    plt.xticks(x_data[::10], x_ticks[::10])
    plt.ylabel("Fatalities")
    plt.title("Real Data vs SIR-Model in " + country_name)
    plt.legend(numpoints=1, loc=2, frameon=None)
    plt.show()

In [ ]:
def new_R0(t, a, b, c):
    return a / (1 + (t/c)**b)

result, country_name, N, D, max_days, CFR_OPT, CFR_scaling_factor, R_0_function, R_0_result_params = fit_extended_SIR("Italy", new_R0, region_name=None, fit_method="least_squares", a=3.0, b=1.5, c=50)
print(result.fit_report())
fitted_plot(result, "Italy");
plot_extended_SIR(*extended_SIR(N, D, max_days, CFR_OPT, CFR_scaling_factor, R_0_function, **R_0_result_params))

In [ ]:
def new_R0(t, a, b, c):
    return a / (1 + (t/c)**b)

result, country_name, N, D, max_days, CFR_OPT, CFR_scaling_factor, R_0_function, R_0_result_params = fit_extended_SIR("Spain", new_R0, region_name=None, fit_method="least_squares", a=3.0, b=1.5, c=50)
print(result.fit_report())
fitted_plot(result, "Spain");
plot_extended_SIR(*extended_SIR(N, D, max_days, CFR_OPT, CFR_scaling_factor, R_0_function, **R_0_result_params))

In [ ]:
def new_R0(t, a, b, c):
    return a / (1 + (t/c)**b)

result, country_name, N, D, max_days, CFR_OPT, CFR_scaling_factor, R_0_function, R_0_result_params = fit_extended_SIR("US", new_R0, region_name=None, fit_method="least_squares", a=3.0, b=1.5, c=50)
print(result.fit_report())
fitted_plot(result, "US");
plot_extended_SIR(*extended_SIR(N, D, max_days, CFR_OPT, CFR_scaling_factor, R_0_function, **R_0_result_params))

In [ ]:
def new_R0(t, a, b, c):
    return a / (1 + (t/c)**b)

result, country_name, N, D, max_days, CFR_OPT, CFR_scaling_factor, R_0_function, R_0_result_params = fit_extended_SIR("Germany", new_R0, region_name=None, fit_method="least_squares", a=3.0, b=1.5, c=50)
print(result.fit_report())
fitted_plot(result, "Germany");
plot_extended_SIR(*extended_SIR(N, D, max_days, CFR_OPT, CFR_scaling_factor, R_0_function, **R_0_result_params))

In [ ]:
def new_R0(t, a, b, c):
    return a / (1 + (t/c)**b)

result, country_name, N, D, max_days, CFR_OPT, CFR_scaling_factor, R_0_function, R_0_result_params = fit_extended_SIR("United Kingdom", new_R0, region_name=None, fit_method="least_squares", a=3.0, b=1.5, c=50)
print(result.fit_report())
fitted_plot(result, "United Kingdom");
plot_extended_SIR(*extended_SIR(N, D, max_days, CFR_OPT, CFR_scaling_factor, R_0_function, **R_0_result_params))

In [ ]:
def new_R0(t, a, b, c):
    return a / (1 + (t/c)**b)

result, country_name, N, D, max_days, CFR_OPT, CFR_scaling_factor, R_0_function, R_0_result_params = fit_extended_SIR("China", new_R0, region_name=None, fit_method="least_squares", a=3.0, b=1.5, c=50)
print(result.fit_report())
fitted_plot(result, "China");
plot_extended_SIR(*extended_SIR(N, D, max_days, CFR_OPT, CFR_scaling_factor, R_0_function, **R_0_result_params))

In [ ]:
def new_R0(t, a, b, c):
    return a / (1 + (t/c)**b)

result, country_name, N, D, max_days, CFR_OPT, CFR_scaling_factor, R_0_function, R_0_result_params = fit_extended_SIR("Iran", new_R0, region_name=None, fit_method="least_squares", a=3.0, b=1.5, c=50)
print(result.fit_report())
fitted_plot(result, "Iran");
plot_extended_SIR(*extended_SIR(N, D, max_days, CFR_OPT, CFR_scaling_factor, R_0_function, **R_0_result_params))

# **3. Prediction** <a id="section3"></a>

## **3.1 Logistic model** <a id="section31"></a>

In [ ]:
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
import scipy.optimize as opt
import os

In [ ]:
submission_example = pd.read_csv("../input/covid19-global-forecasting-week-4/submission.csv")
test = pd.read_csv("../input/covid19-global-forecasting-week-4/test.csv")
train = pd.read_csv("../input/covid19-global-forecasting-week-4/train.csv")

In [ ]:
train_ = train[train["ConfirmedCases"] >= 0]
train_.head()

Replace all na (Province_State) with country:

In [ ]:
EMPTY_VAL = "EMPTY_VAL"

def fillState(state, country):
    if state == EMPTY_VAL: return country
    return state

train_['Province_State'].fillna(EMPTY_VAL, inplace=True)
train_['Province_State'] = train_.loc[:, ['Province_State', 'Country_Region']].apply(lambda x : fillState(x['Province_State'], x['Country_Region']), axis=1)
test['Province_State'].fillna(EMPTY_VAL, inplace=True)
test['Province_State'] = test.loc[:, ['Province_State', 'Country_Region']].apply(lambda x : fillState(x['Province_State'], x['Country_Region']), axis=1)
test.head()

In [ ]:
train_['row_number'] = train_.groupby(['Country_Region', 'Province_State']).cumcount()
x = train_[train_["Country_Region"] == 'US'][train_["Province_State"] == 'New York']['row_number']
y = train_[train_["Country_Region"] == 'US'][train_["Province_State"] == 'New York']['ConfirmedCases']
y_ = train_[train_["Country_Region"] == 'US'][train_["Province_State"] == 'New York']['Fatalities']

def f(x, L, b, k, x_0):
    return L / (1. + np.exp(-k * (x - x_0))) + b


def logistic(xs, L, k, x_0):
    result = []
    for x in xs:
        xp = k*(x-x_0)
        if xp >= 0:
            result.append(L / ( 1. + np.exp(-xp) ) )
        else:
            result.append(L * np.exp(xp) / ( 1. + np.exp(xp) ) )
    return result

p0 = [max(y), 0.0,max(x)]
p0_ = [max(y_), 0.0,max(x)]
x_ = np.arange(0, 100, 1).tolist()
try:
    popt, pcov = opt.curve_fit(logistic, x, y,p0)
    yfit = logistic(x_, *popt)
    popt_, pcov_ = opt.curve_fit(logistic, x, y_,p0_)
    yfit_ = logistic(x_, *popt_)
except:
    popt, pcov = opt.curve_fit(f, x, y, method="lm", maxfev=5000)
    yfit = f(x_, *popt)
    popt_, pcov_ = opt.curve_fit(f, x, y_, method="lm", maxfev=5000)
    yfit_ = f(x_, *popt_)
    #print("problem")


fig, ax = plt.subplots(1, 1, figsize=(10, 8))
ax.plot(x, y, 'o', label ='Actual Cases')
ax.plot(x_, yfit, '-', label ='Fitted Cases')

ax.plot(x, y_, 'o', label ='Actual Fatalities')
ax.plot(x_, yfit_, '-', label ='Fitted fatalities')
ax.title.set_text('US - New York')
plt.legend(loc="center right")
plt.show()

correlation_matrix = np.corrcoef(y[1:73], yfit[1:73])
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2
r_squared

correlation_matrix = np.corrcoef(y_[1:73], yfit_[1:73])
correlation_xy = correlation_matrix[0,1]
r_squared1 = correlation_xy**2
r_squared1

print("The R^2 for the Actual Cases is:", r_squared,"\n\n The R^2 for the Fatalities is:",r_squared1)

In [ ]:
train_['row_number'] = train_.groupby(['Country_Region', 'Province_State']).cumcount()
x = train_[train_["Country_Region"] == 'Italy'][train_["Province_State"] == 'Italy']['row_number']
y = train_[train_["Country_Region"] == 'Italy'][train_["Province_State"] == 'Italy']['ConfirmedCases']
y_ = train_[train_["Country_Region"] == 'Italy'][train_["Province_State"] == 'Italy']['Fatalities']

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
ax.plot(x, y, 'o', label ='Actual Cases')
ax.plot(x_, yfit, '-', label ='Fitted Cases')

ax.plot(x, y_, 'o', label ='Actual Fatalities')
ax.plot(x_, yfit_, '-', label ='Fitted fatalities')
ax.title.set_text('Italy')
plt.legend(loc="center right")
plt.show()

correlation_matrix = np.corrcoef(y[1:73], yfit[1:73])
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2
r_squared

correlation_matrix = np.corrcoef(y_[1:73], yfit_[1:73])
correlation_xy = correlation_matrix[0,1]
r_squared1 = correlation_xy**2
r_squared1

print("The R^2 for the Actual Cases is:", r_squared,"\n\n The R^2 for the Fatalities is:",r_squared1)

In [ ]:
train_['row_number'] = train_.groupby(['Country_Region', 'Province_State']).cumcount()
x = train_[train_["Country_Region"] == 'Spain'][train_["Province_State"] == 'Spain']['row_number']
y = train_[train_["Country_Region"] == 'Spain'][train_["Province_State"] == 'Spain']['ConfirmedCases']
y_ = train_[train_["Country_Region"] == 'Spain'][train_["Province_State"] == 'Spain']['Fatalities']

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
ax.plot(x, y, 'o', label ='Actual Cases')
ax.plot(x_, yfit, '-', label ='Fitted Cases')

ax.plot(x, y_, 'o', label ='Actual Fatalities')
ax.plot(x_, yfit_, '-', label ='Fitted fatalities')
ax.title.set_text('Spain')
plt.legend(loc="center right")
plt.show()

correlation_matrix = np.corrcoef(y[1:73], yfit[1:73])
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2
r_squared

correlation_matrix = np.corrcoef(y_[1:73], yfit_[1:73])
correlation_xy = correlation_matrix[0,1]
r_squared1 = correlation_xy**2
r_squared1

print("The R^2 for the Actual Cases is:", r_squared,"\n\n The R^2 for the Fatalities is:",r_squared1)

In [ ]:
train_['row_number'] = train_.groupby(['Country_Region', 'Province_State']).cumcount()
x = train_[train_["Country_Region"] == 'Germany'][train_["Province_State"] == 'Germany']['row_number']
y = train_[train_["Country_Region"] == 'Germany'][train_["Province_State"] == 'Germany']['ConfirmedCases']
y_ = train_[train_["Country_Region"] == 'Germany'][train_["Province_State"] == 'Germany']['Fatalities']

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
ax.plot(x, y, 'o', label ='Actual Cases')
ax.plot(x_, yfit, '-', label ='Fitted Cases')

ax.plot(x, y_, 'o', label ='Actual Fatalities')
ax.plot(x_, yfit_, '-', label ='Fitted fatalities')
ax.title.set_text('Germany')
plt.legend(loc="center right")
plt.show()

correlation_matrix = np.corrcoef(y[1:73], yfit[1:73])
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2
r_squared

correlation_matrix = np.corrcoef(y_[1:73], yfit_[1:73])
correlation_xy = correlation_matrix[0,1]
r_squared1 = correlation_xy**2
r_squared1

print("The R^2 for the Actual Cases is:", r_squared,"\n\n The R^2 for the Fatalities is:",r_squared1)

In [ ]:
train_['row_number'] = train_.groupby(['Country_Region', 'Province_State']).cumcount()
x = train_[train_["Country_Region"] == 'United Kingdom'][train_["Province_State"] == 'United Kingdom']['row_number']
y = train_[train_["Country_Region"] == 'United Kingdom'][train_["Province_State"] == 'United Kingdom']['ConfirmedCases']
y_ = train_[train_["Country_Region"] == 'United Kingdom'][train_["Province_State"] == 'United Kingdom']['Fatalities']

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
ax.plot(x, y, 'o', label ='Actual Cases')
ax.plot(x_, yfit, '-', label ='Fitted Cases')

ax.plot(x, y_, 'o', label ='Actual Fatalities')
ax.plot(x_, yfit_, '-', label ='Fitted fatalities')
ax.title.set_text('United Kingdom')
plt.legend(loc="center right")
plt.show()

correlation_matrix = np.corrcoef(y[1:73], yfit[1:73])
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2
r_squared

correlation_matrix = np.corrcoef(y_[1:73], yfit_[1:73])
correlation_xy = correlation_matrix[0,1]
r_squared1 = correlation_xy**2
r_squared1

print("The R^2 for the Actual Cases is:", r_squared,"\n\n The R^2 for the Fatalities is:",r_squared1)

In [ ]:
train_['row_number'] = train_.groupby(['Country_Region', 'Province_State']).cumcount()
x = train_[train_["Country_Region"] == 'Iran'][train_["Province_State"] == 'Iran']['row_number']
y = train_[train_["Country_Region"] == 'Iran'][train_["Province_State"] == 'Iran']['ConfirmedCases']
y_ = train_[train_["Country_Region"] == 'Iran'][train_["Province_State"] == 'Iran']['Fatalities']

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
ax.plot(x, y, 'o', label ='Actual Cases')
ax.plot(x_, yfit, '-', label ='Fitted Cases')

ax.plot(x, y_, 'o', label ='Actual Fatalities')
ax.plot(x_, yfit_, '-', label ='Fitted fatalities')
ax.title.set_text('Iran')
plt.legend(loc="center right")
plt.show()

correlation_matrix = np.corrcoef(y[1:73], yfit[1:73])
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2
r_squared

correlation_matrix = np.corrcoef(y_[1:73], yfit_[1:73])
correlation_xy = correlation_matrix[0,1]
r_squared1 = correlation_xy**2
r_squared1

print("The R^2 for the Actual Cases is:", r_squared,"\n\n The R^2 for the Fatalities is:",r_squared1)

## **3.2. Gompertz model** <a id="section32"></a>

In [ ]:
from itertools import cycle, islice
import seaborn as sb
import matplotlib.dates as dates
import datetime as dt

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from itertools import cycle, islice

import plotly.offline as py
py.init_notebook_mode(connected=True)
from plotly import tools, subplots
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go

from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.pipeline import make_pipeline
from tqdm import tqdm

from scipy.optimize.minpack import curve_fit
from sklearn.metrics import r2_score
from scipy.special import expit

In [ ]:
test_data = pd.read_csv("../input/covid19-global-forecasting-week-4/test.csv")
train_data = pd.read_csv("../input/covid19-global-forecasting-week-4/train.csv")

In [ ]:
train_data['NewConfirmedCases'] = train_data['ConfirmedCases'] - train_data['ConfirmedCases'].shift(1)
train_data['NewConfirmedCases'] = train_data['NewConfirmedCases'].fillna(0.0)
train_data['NewFatalities']     = train_data['Fatalities'] - train_data['Fatalities'].shift(1)
train_data['NewFatalities']     = train_data['NewFatalities'].fillna(0.0)#.astype(int)

train_data.head()

In [ ]:
def getColumnInfo(df):
    n_province =  df['Province_State'].nunique()
    n_country  =  df['Country_Region'].nunique()
    n_days     =  df['Date'].nunique()
    start_date =  df['Date'].unique()[0]
    end_date   =  df['Date'].unique()[-1]
    return n_province, n_country, n_days, start_date, end_date

n_train = train_data.shape[0]
n_test = test_data.shape[0]

n_prov_train, n_count_train, n_train_days, start_date_train, end_date_train = getColumnInfo(train_data)
n_prov_test,  n_count_test,  n_test_days,  start_date_test,  end_date_test  = getColumnInfo(test_data)
df_test = test_data.loc[test_data.Date > '2020-04-03']
overlap_days = n_test_days - df_test.Date.nunique()

prob_confirm_check_train = train_data.ConfirmedCases.value_counts(normalize=True)
prob_fatal_check_train = train_data.Fatalities.value_counts(normalize=True)

n_confirm_train = train_data.ConfirmedCases.value_counts()[1:].sum()
n_fatal_train = train_data.Fatalities.value_counts()[1:].sum()

In [ ]:
#train_data_by_country = train_data.groupby(['Country_Region'],as_index=True).agg({'ConfirmedCases': 'max', 'Fatalities': 'max'})
#train_data_by_country_confirm = train_data_by_country.sort_values(by=["ConfirmedCases"], ascending=False)
train_data_by_country = train_data.groupby(['Date','Country_Region'],as_index=False).agg({'ConfirmedCases': 'sum', 'Fatalities': 'sum'
                                                                                         })
                                                       #'GrowthRate':'mean'
max_train_date = train_data['Date'].max()
train_data_by_country_confirm = train_data_by_country.query('(Date == @max_train_date) & (ConfirmedCases > 100)').sort_values('ConfirmedCases', ascending=False)
train_data_by_country_confirm.set_index('Country_Region', inplace=True)
display(train_data_by_country_confirm.head())

from itertools import cycle, islice
discrete_col = list(islice(cycle(['orange', 'r', 'g', 'k', 'b', 'c', 'm']), None, len(train_data_by_country_confirm.head(30))))
plt.rcParams.update({'font.size': 22})
train_data_by_country_confirm.head(20).plot(figsize=(20,15), kind='barh', color=discrete_col)
plt.legend(["Confirmed Cases", "Fatalities"]);
plt.xlabel("Number of Covid-19 Affectees")
plt.title("First 20 Countries with Highest Confirmed Cases")
ylocs, ylabs = plt.yticks()
for i, v in enumerate(train_data_by_country_confirm.head(20)["ConfirmedCases"][:]):
    plt.text(v+0.01, ylocs[i]-0.25, str(int(v)), fontsize=12)
for i, v in enumerate(train_data_by_country_confirm.head(20)["Fatalities"][:]):
    if v > 0: #disply for only >300 fatalities
        plt.text(v+0.01,ylocs[i]+0.1,str(int(v)),fontsize=12)  

In [ ]:
''''#train_data_by_country = train_data.groupby(['Country_Region'],as_index=True).agg({'ConfirmedCases': 'max', 'Fatalities': 'max'})
#train_data_by_country_confirm = train_data_by_country.sort_values(by=["ConfirmedCases"], ascending=False)

train_data['Date'] = pd.to_datetime(train_data['Date'])
train_data_by_date = train_data.groupby(['Date'],as_index=True).agg({'ConfirmedCases': 'max','Fatalities': 'max', 
                                                                     'NewConfirmedCases':'max', 'NewFatalities':'max'})

## ======= Sort by countries with fatalities > 200 ========
train_data_by_country_fatal = train_data_by_country[train_data_by_country['Fatalities']>200]
train_data_by_country_fatal = train_data_by_country_fatal.sort_values(by=['Fatalities'],ascending=False).reset_index()
#display(train_data_by_country_fatal.head(20))

df_merge_by_country = pd.merge(train_data,train_data_by_country_fatal['Country_Region'],on=['Country_Region'],how='inner')
df_max_fatality_country = df_merge_by_country.groupby(['Date','Country_Region'],as_index=False).agg({'ConfirmedCases': 'sum',
                                                                                                     'Fatalities': 'sum',
                                                                                                     'NewConfirmedCases':'sum',
                                                                                                     'NewFatalities':'sum'})


# convert -ve numbers to zeros, and set Date as new index
num = df_max_fatality_country._get_numeric_data() 
num[num < 0.0] = 0.0
df_max_fatality_country.set_index('Date',inplace=True)
#display(df_max_fatality_country.head(20))

countries = train_data_by_country_fatal['Country_Region'].unique()''''''


   

In [ ]:
def reformat_time(reformat, ax):
    ax.xaxis.set_major_locator(dates.WeekdayLocator())
    ax.xaxis.set_major_formatter(dates.DateFormatter('%b %d'))    
    if reformat: #reformat again if you wish
        date_list = train_data_by_date.reset_index()["Date"].tolist()
        x_ticks = [dt.datetime.strftime(t,'%Y-%m-%d') for t in date_list]
        x_ticks = [tick for i,tick in enumerate(x_ticks) if i%8==0 ]# split labels into same number of ticks as by pandas
        ax.set_xticklabels(x_ticks, rotation=90)
    # cosmetics
    ax.yaxis.grid(linestyle='dotted')
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.spines['left'].set_color('none')
    ax.spines['bottom'].set_color('none')

train_data['Date'] = pd.to_datetime(train_data['Date'])
train_data_by_date = train_data.groupby(['Date'],as_index=True).agg({'ConfirmedCases': 'sum','Fatalities': 'sum', 
                                                                     'NewConfirmedCases':'sum', 'NewFatalities':'sum'})
#                                             MortalityRate':'mean'
num0 = train_data_by_date._get_numeric_data() 
num0[num0 < 0.0] = 0.0
#display(train_data_by_date.head())

## ======= Sort by countries with fatalities > 500 ========

train_data_by_country_max = train_data.groupby(['Country_Region'],as_index=True).agg({'ConfirmedCases': 'max', 'Fatalities': 'max'})
train_data_by_country_fatal = train_data_by_country_max[train_data_by_country_max['Fatalities']>500]
train_data_by_country_fatal = train_data_by_country_fatal.sort_values(by=['Fatalities'],ascending=False).reset_index()
display(train_data_by_country_fatal.head(20))

df_merge_by_country = pd.merge(train_data,train_data_by_country_fatal['Country_Region'],on=['Country_Region'],how='inner')
df_max_fatality_country = df_merge_by_country.groupby(['Date','Country_Region'],as_index=False).agg({'ConfirmedCases': 'sum',
                                                                                                     'Fatalities': 'sum',
                                                                                                     'NewConfirmedCases':'sum',
                                                                                                     'NewFatalities':'sum'
                                                                                                     })
#                                               MortalityRate':'mean'

num1 = df_max_fatality_country._get_numeric_data() 
num1[num1 < 0.0] = 0.0
df_max_fatality_country.set_index('Date',inplace=True)
#display(df_max_fatality_country.head(20))

countries = train_data_by_country_fatal['Country_Region'].unique()

plt.rcParams.update({'font.size': 16})

fig,(ax0,ax1) = plt.subplots(1,2,figsize=(15, 8))
fig,(ax2,ax3) = plt.subplots(1,2,figsize=(15, 8))#,sharey=True)

train_data_by_date.ConfirmedCases.plot(ax=ax0, x_compat=True, title='Confirmed Cases Globally', legend='Confirmed Cases',
                                       color=discrete_col)#, logy=True)
reformat_time(0,ax0)
train_data_by_date.NewConfirmedCases.plot(ax=ax0, x_compat=True, linestyle='dotted', legend='New Confirmed Cases',
                                          color=discrete_col)#, logy=True)
reformat_time(0,ax0)

train_data_by_date.Fatalities.plot(ax=ax2, x_compat=True, title='Fatalities Globally', legend='Fatalities', color='r')
reformat_time(0,ax2)
train_data_by_date.NewFatalities.plot(ax=ax2, x_compat=True, linestyle='dotted', legend='Daily Deaths',color='r')#tell pandas not to use its own datetime format
reformat_time(0,ax2)

for country in countries:
    match = df_max_fatality_country.Country_Region==country
    df_fatality_by_country = df_max_fatality_country[match] 
    df_fatality_by_country.ConfirmedCases.plot(ax=ax1, x_compat=True, title='Cumulative Confirmed Cases Nationally')
    reformat_time(0,ax1)
    df_fatality_by_country.Fatalities.plot(ax=ax3, x_compat=True, title='Cumulative Fatalities Nationally')
    reformat_time(0,ax3)
    
ax1.legend(countries)
ax3.legend(countries)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.pipeline import make_pipeline
from tqdm import tqdm

plt.rcParams.update({'font.size': 12})
fig,(ax0,ax1) = plt.subplots(2,1,figsize=(20, 20))
countries_europe = ['Italy', 'France', 'Spain', 'Germany', 'United Kingdom']

# Take the 1st day as 2020-02-23
df = train_data.loc[train_data.Date >= '2020-02-23']
n_days_europe = df.Date.nunique()

for country in tqdm(countries): 
    df_country_train = df_max_fatality_country[df_max_fatality_country['Country_Region']==country] 
    df_country_test = test_data[test_data['Country_Region']==country]  
    df_country_train = df_country_train.reset_index()[df_country_train.reset_index().Date > '2020-02-22']
    
    x_train = np.arange(1, n_days_europe+1).reshape((-1,1))
    x_test  = (np.arange(1,n_test_days+1+overlap_days)).reshape((-1,1)) 
    #print (range(n_days_europe))
    
    y_train_f = df_country_train['Fatalities']
    #print(y_train_f)
    model_f = make_pipeline(PolynomialFeatures(degree=3), Ridge(fit_intercept=False)) 
    model_f = model_f.fit(x_train, y_train_f)
    y_predict_f = model_f.predict(x_test) 
    
    y_train_c = df_country_train['ConfirmedCases'] 
    model_c = make_pipeline(PolynomialFeatures(degree=3), Ridge(fit_intercept=False)) 
    model_c = model_c.fit(x_train, y_train_c)
    y_predict_c = model_c.predict(x_test)
    
    ax0.plot(x_test, y_predict_c,linewidth=2, label='predict_'+country)
    ax0.plot(x_train, y_train_c, linewidth=2, color='r', linestyle='dotted', label='train_'+country)
    ax0.set_title("Prediction vs Training for Confirmed Cases")
    ax0.set_xlabel("Number of days")
    ax0.set_ylabel("Confirmed Cases")
    ax0.legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
    
    ax1.plot(x_test, y_predict_f,linewidth=2, label='predict_'+country)
    ax1.plot(x_train, y_train_f, linewidth=2, color='r', linestyle='dotted', label='train_'+country)
    ax1.set_title("Prediction vs Training for Fatalities")
    ax1.set_xlabel("Number of days")
    ax1.set_ylabel("Fatalities")
    ax1.legend(loc='center left',bbox_to_anchor=(1.0, 0.5))

In [ ]:
#only selected countries
#countries= countries[0:9]
from tqdm import tqdm
from scipy.optimize.minpack import curve_fit
from sklearn.metrics import r2_score
from scipy.special import expit

def Gompertz(a, c, t, t0):    
    Q = a * np.exp(-np.exp(-c*(t-t0)))
    return Q
def Boltzman(a, c, t, t0):
    Q = a / (1 + np.exp(-c*(t-t0)))
    return Q

plt.rcParams.update({'font.size': 12})
fig,(ax0,ax1) = plt.subplots(2,1,figsize=(20, 20))
#countries_europe=['US', 'China', 'Iran', 'France', 'Italy', 'Spain', 'Germany', 'Belgium', 'Turkey', 'Netherlands', 'Switzerland', 'United Kingdom']
#countries_europe=['France']
for country in tqdm(countries): 
    #print('\n\n\n\n country ==>', country)
    df_country_train = df_max_fatality_country[df_max_fatality_country['Country_Region']==country] 
    df_country_test = test_data[test_data['Country_Region']==country]  
    if country != 'China':
        df_country_train = df_country_train.reset_index().loc[df_country_train.reset_index().Date>'2020-02-22'] #17
        n_days_sans_China =train_data.Date.nunique() - df_country_train.Date.nunique()        
    else:
        df_country_train = df_country_train.reset_index()
        n_days_sans_China = 0
        
    n_train_days =df_country_train.Date.nunique()    
    x_train = range(n_train_days)
    x_test  = range(n_train_days+n_test_days-overlap_days)#n_test_days+overlap_days)
    y_train_f = df_country_train['Fatalities']
    y_train_c = df_country_train['ConfirmedCases']    
    
    if country == 'China':
        lower = [100, 0.02, 0]
        upper = [2.0*max(y_train_f),0.2, 40]
    elif country == 'Iran':
        lower = [200, 0.00, 0]
        upper = [3.0*max(y_train_f),0.14, 65]
    elif country == 'Italy':
        lower = [100, 0.00, 0]
        upper = [3.5*max(y_train_f),0.15, 72]
    elif country == 'US':
        lower = [0, 0.02, 0]
        upper = [4.0*max(y_train_f),0.22, 83] 
    elif country == 'France':
        lower = [0, 0.02, 0]
        upper = [4.0*max(y_train_f),0.18, 82]    
    elif country == 'Spain':
        lower = [0, 0.02, 0]
        upper = [3.5*max(y_train_f),0.18, 75]
    elif country == 'Germany':
        lower = [0.0, 0.02, 0]
        upper = [3.5*max(y_train_f),0.25, 80] 
    elif country == 'Belgium':
        lower = [0.0, 0.02, 0]
        upper = [3.5*max(y_train_f),0.25, 80] 
    elif country == 'Turkey':
        lower = [0.0, 0.02, 0]
        upper = [4.0*max(y_train_f),0.25, 83]
    elif country == 'Netherlands':
        lower = [0.0, 0.02, 0]
        upper = [4.0*max(y_train_f),0.18, 80] 
    elif country == 'Switzerland':
        lower = [0.0, 0.02, 0]
        upper = [4.0*max(y_train_f),0.18, 80] 
    elif country == 'United Kingdom':
        lower = [0.0, 0.02, 0]
        upper = [4.5*max(y_train_f),0.25, 85]     
    else:
        lower = [0.0, 0.02, 0]
        upper = [4.5*max(y_train_f),0.25, 80]    
    
    popt_f, pcov_f = curve_fit(Gompertz, x_train, y_train_f, method='trf', bounds=(lower,upper))
    a_max, estimated_c, estimated_t0 = popt_f
    y_predict_f = Gompertz(a_max, estimated_c, x_test, estimated_t0)
    y_predict_f_at_t0 =  Gompertz(a_max, estimated_c, estimated_t0, estimated_t0)
    print("\n\n\n\n",country,'\nfatalities ==>, max: ',a_max, ', slope: %.2f'% estimated_c, ', inflection point: ', 
          estimated_t0, ', r2 score: %.2f'% r2_score(y_train_f[:], y_predict_f[0:n_train_days]))
   
    ###### Confirmed cases:    
        
    if country == 'China':
        lower_c = [100, 0.02, 0]
        upper_c = [2.0*max(y_train_c),0.25,30]
    elif country == 'Iran':
        lower_c = [100, 0.00, 0]
        upper_c = [3.0*max(y_train_c),0.15,65]
    elif country == 'Italy':
        lower_c = [1000, 0.00, 0]
        upper_c = [3.0*max(y_train_c),0.15, 64]
    elif country == 'US':
        lower_c = [0, 0.02, 0]
        upper_c = [3.5*max(y_train_c),0.23, 78] 
    elif country == 'France':
        lower_c = [10, 0.02, 0]
        upper_c = [4.5*max(y_train_c),0.15, 85]
    elif country == 'Spain':
        lower_c = [10, 0.02, 0]
        upper_c = [3.5*max(y_train_c),0.15, 74] 
    elif country == 'Germany':
        lower_c = [10, 0.02, 0]
        upper_c = [3.5*max(y_train_c),0.15, 75] 
    elif country == 'Belgium':
        lower_c = [100, 0.02, 0]
        upper_c = [4.0*max(y_train_c),0.15, 80]
    elif country == 'Turkey':
        lower_c = [100, 0.02, 0]
        upper_c = [4.0*max(y_train_c),0.25, 83] 
    elif country == 'Netherlands':
        lower_c = [100, 0.02, 0]
        upper_c = [4.0*max(y_train_c),0.14, 78] 
    elif country == 'Switzerland':
        lower_c = [100, 0.02, 0]
        upper_c = [3.5*max(y_train_c),0.14, 70]  
    elif country == 'United Kingdom':
        lower_c = [100, 0.02, 0]
        upper_c = [4.5*max(y_train_c),0.15, 85]    
    else:
        lower_c = [100, 0.02, 0]
        upper_c = [4.5*max(y_train_c),0.15, 80]
        
    
    popt_c, pcov_c = curve_fit(Gompertz, x_train, y_train_c, method='trf', bounds=(lower_c,upper_c))
    a_max_c, estimated_c_c, estimated_t0_c = popt_c
    y_predict_c = Gompertz(a_max_c, estimated_c_c, x_test, estimated_t0_c)
    y_predict_c_at_t0 =  Gompertz(a_max_c, estimated_c_c, estimated_t0_c, estimated_t0_c)
    print('confirmed ==> max: ',a_max_c, ', slope: %.2f'% estimated_c_c, ', inflection point: ', 
          estimated_t0_c, ', r2 score: %.2f'% r2_score(y_train_c[:], y_predict_c[0:n_train_days]))
    
    ## ===== Move the x-axis of trained and test datasets to allign with dates in China ======
    extend_days_test = [i+len(x_test) for i in range(n_days_sans_China)]
    x_test      = np.append(x_test, extend_days_test) 
    y_predict_c = np.pad(y_predict_c, (n_days_sans_China, 0), 'constant')
    y_predict_f = np.pad(y_predict_f, (n_days_sans_China, 0), 'constant')
    inflection_c = estimated_t0_c+n_days_sans_China

    extend_days_train = [i+len(x_train) for i in range(n_days_sans_China)]
    x_train     = np.append(x_train, extend_days_train)
    y_train_c   = np.pad(y_train_c, (n_days_sans_China, 0), 'constant')
    y_train_f   = np.pad(y_train_f, (n_days_sans_China, 0), 'constant')
    inflection_f = estimated_t0+n_days_sans_China
    
    ## ===== Plot =======
    ax0.plot(x_test, y_predict_c, linewidth=2, label='predict_'+country) 
    ax0.plot(inflection_c, y_predict_c_at_t0, marker='o', markersize=6, color='green')#, label='inflection')
    ax0.plot(x_train, y_train_c, linewidth=2, color='r', linestyle='dotted', label='train_'+country)   
    ax0.set_title("Prediction vs Training for Confirmed Cases")
    ax0.set_xlabel("Number of days")
    ax0.set_ylabel("Confirmed Cases")
    ax0.legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
    
    
    ax1.plot(x_test, y_predict_f, linewidth=2, label='predict_'+country) 
    ax1.plot(inflection_f, y_predict_f_at_t0, marker='o', markersize=6, color='green')
    ax1.plot(x_train, y_train_f, linewidth=2, color='r', linestyle='dotted', label='train_'+country)    
    ax1.set_title("Prediction vs Training for Fatalities")
    ax1.set_xlabel("Number of days")
    ax1.set_ylabel("Fatalities")
    ax1.legend(loc='center left',bbox_to_anchor=(1.0, 0.5))

## **3.3. Final submission using Gompertz model**

In [ ]:
nCountries= train_data['Country_Region'].unique() 
isState = bool

def get_bounds_fatal (country, isState, y_train):
    maximum = max(y_train)
    if maximum == 0.0: maximum = 1.0 

    if country == 'China':
        lower = [0.0, 0.02, 0]
        upper = [2.0*maximum,0.2, 40]
    elif country == 'Iran':
        lower = [200, 0.00, 0]
        upper = [3.0*maximum,0.14, 65]
    elif country == 'Italy':
        lower = [100, 0.00, 0]
        upper = [3.5*maximum,0.15, 70]
    elif country == 'US':
        lower = [0, 0.02, 0]
        if maximum <=10:upper = [4.0*maximum, 0.30, 85] 
        else:           upper = [4.0*maximum,0.23, 85] 
    elif country == 'France':
        lower = [0, 0.02, 0]
        if maximum <=10:upper = [4.0*maximum,0.18, 80]
        else:           upper = [4.0*maximum,0.18, 80] 
    elif country == 'Spain':
        lower = [0, 0.02, 0]
        upper = [3.5*maximum,0.18, 75]
    elif country == 'Germany':
        lower = [0.0, 0.02, 0]
        upper = [3.5*maximum,0.25, 80] 
    elif country == 'Belgium':
        lower = [0.0, 0.02, 0]
        upper = [3.5*maximum,0.25, 80] 
    elif country == 'Turkey':
        lower = [0.0, 0.02, 0]
        upper = [4.0*maximum,0.25, 83]
    elif country == 'Netherlands':
        lower = [0.0, 0.02, 0]
        upper = [4.0*maximum,0.18, 80] 
    elif country == 'Switzerland':
        lower = [0.0, 0.02, 0]
        upper = [4.0*maximum,0.18, 80] 
    elif country == 'United Kingdom':
        lower = [0.0, 0.02, 0]
        upper = [4.5*maximum,0.25, 85] 
    elif country == 'Denmark':
        lower = [0, 0.02, 0]
        if maximum <=10:upper = [4.0*maximum, 0.30, 80] 
        else:           upper = [4.0*maximum,0.23, 80]  
    elif country == 'Australia':
        lower = [0, 0.02, 0]
        if maximum <=10: upper = [2.0*maximum, 0.20, 45] 
        else:            upper = [2.5*maximum,0.20, 65]  
    elif country == 'Canada':
        lower = [0, 0.02, 0]
        if maximum <=10: upper = [2.0*maximum, 0.20, 65] 
        else:            upper = [3.5*maximum, 0.30, 80] 
    elif country == 'Pakistan':
        lower = [0.0, 0.02, 0] 
        upper = [4.5*maximum,0.20,85]   
    elif country == 'India':
        lower = [0.0, 0.02, 0] 
        upper = [4.5*maximum,0.25,85]       
    else:
        lower = [0.0, 0.02, 0] 
        if isState:
            if maximum <=10:upper = [4.0*maximum,0.30,80] 
            else:           upper = [4.5*maximum,0.15,80]
        else: 
            if maximum <=10:upper = [4.0*maximum,0.60,85] 
            else:           upper = [4.5*maximum,0.25,85]    
    return lower, upper 

def get_bounds_confirm (country, isState, y_train):
    maximum = max(y_train)
    if maximum == 0.0: maximum = 1.0

    if country == 'China':
        lower_c = [0, 0.02, 0]
        upper_c = [2.0*maximum,0.25,30]
    elif country == 'Iran':
        lower_c = [100, 0.00, 0]
        upper_c = [3.0*maximum,0.15,65]
    elif country == 'Italy':
        lower_c = [1000, 0.00, 0]
        upper_c = [3.0*maximum,0.15, 64]
    elif country == 'US':
        lower_c = [0, 0.02, 0]
        if maximum <=10:upper_c = [4.0*maximum, 0.30, 80] 
        else:           upper_c = [3.5*maximum, 0.23, 78] 
    elif country == 'France':
        lower_c = [10, 0.02, 0]
        if maximum <=10:upper_c = [4.0*maximum, 0.15, 80] 
        else:           upper_c = [4.5*maximum, 0.15, 80] 
    elif country == 'Spain':
        lower_c = [10, 0.02, 0]
        upper_c = [3.5*maximum,0.15, 74] 
    elif country == 'Germany':
        lower_c = [10, 0.02, 0]
        upper_c = [3.5*maximum,0.15, 75] 
    elif country == 'Belgium':
        lower_c = [100, 0.02, 0]
        upper_c = [4.0*maximum,0.15, 80]
    elif country == 'Turkey':
        lower_c = [100, 0.02, 0]
        upper_c = [4.0*maximum,0.25, 83] 
    elif country == 'Netherlands':
        lower_c = [0.0, 0.02, 0]
        upper_c = [4.0*maximum,0.14, 78] 
    elif country == 'Switzerland':
        lower_c = [100, 0.02, 0]
        upper_c = [3.5*maximum,0.14, 70] 
    elif country == 'United Kingdom':
        lower_c = [0.0, 0.02, 0]
        upper_c = [4.5*maximum,0.15, 85] 
    elif country == 'Denmark':
        lower_c = [0, 0.02, 0]
        if maximum <=10: upper_c = [2.0*maximum, 0.30, 40] 
        else:            upper_c = [2.5*maximum,0.30, 55]   
    elif country == 'Australia':
        lower_c = [0, 0.02, 0]
        if maximum <=10: upper_c = [2.0*maximum, 0.25, 45] 
        else:            upper_c = [2.5*maximum,0.25, 65]  
    elif country == 'Canada':
        lower_c = [0, 0.02, 0]
        if maximum <=10: upper_c = [3.0*maximum, 0.28, 75] 
        else:            upper_c = [3.5*maximum,0.28, 80]
    elif country == 'Pakistan':
        lower_c = [0.0, 0.02, 0] 
        upper_c = [4.5*maximum,0.15,85] 
    elif country == 'India':
        lower_c = [0.0, 0.02, 0] 
        upper_c = [4.5*maximum,0.20,85]     
    else:
        lower_c = [0.0, 0.02, 0] 
        if isState:
            if maximum <= 200: upper_c = [2.0*maximum,0.20,80] 
            else:              upper_c = [4.5*maximum,0.20,80]
        else:  
            if maximum <= 200: upper_c = [3.0*maximum,0.20,85]  
            else:              upper_c = [4.5*maximum,0.15,80]    
    return lower_c, upper_c 

#nCountries = ['United Kingdom']  
x_train = range(n_train_days)
x_test  = range(n_train_days+n_test_days-overlap_days)

for country in tqdm(nCountries): 
    fig,(ax0,ax1) = plt.subplots(2,1,figsize=(20, 20))
    #print('\n\n\n\n country ==>', country) 
    
    df_country_train = train_data[train_data['Country_Region']==country] 
    df_country_test = test_data[test_data['Country_Region']==country]  
    
    if country != 'China':
        df_country_train = df_country_train.reset_index().loc[df_country_train.reset_index().Date>'2020-02-22'] #17
        n_days_sans_China =train_data.Date.nunique() - df_country_train.Date.nunique()        
    else:
        df_country_train = df_country_train.reset_index()
        n_days_sans_China = 0
        
    n_train_days =df_country_train.Date.nunique()    
    x_train = range(n_train_days)
    x_test  = range(n_train_days+n_test_days-overlap_days)   
    nvalues = df_country_train['Province_State'].isna().nunique() #fix for problem with Denmark data
    
    if (df_country_train['Province_State'].isna().unique()==True).any() and nvalues<2: 
        isState = False        
        y_train_f = df_country_train['Fatalities']
        y_train_c = df_country_train['ConfirmedCases']  
        
        if y_train_f.empty == False:
            lower, upper = get_bounds_fatal (country, isState, y_train_f)
            popt_f, pcov_f = curve_fit(Gompertz, x_train, y_train_f, method='trf', bounds=(lower,upper))
            a_max, estimated_c, estimated_t0 = popt_f
            y_predict_f = Gompertz(a_max, estimated_c, x_test, estimated_t0)            
            #print('\nfatalities ==>, max: ',a_max, ', slope: %.2f'% estimated_c, ', inflection point: ', 
             #     estimated_t0, ', r2 score: %.2f'% r2_score(y_train_f[:], y_predict_f[0:n_train_days]))
            
            
        if y_train_c.empty == False:  
            lower_c, upper_c = get_bounds_confirm (country, isState, y_train_c)
            popt_c, pcov_c = curve_fit(Gompertz, x_train, y_train_c, method='trf', bounds=(lower_c,upper_c))
            a_max_c, estimated_c_c, estimated_t0_c = popt_c
            y_predict_c = Gompertz(a_max_c, estimated_c_c, x_test, estimated_t0_c)
            #print('\nconfirmed ==> max: ',a_max_c, ', slope: %.2f'% estimated_c_c, ', inflection point: ', 
             #     estimated_t0_c, ', r2 score: %.2f'% r2_score(y_train_c[:], y_predict_c[0:n_train_days]))
            
            
        ## ===== Move the x-axis of trained and test datasets to allign with dates in China ======
        extend_days_test = [i+len(x_test) for i in range(n_days_sans_China)]
        x_test      = np.append(x_test, extend_days_test)                         
        y_predict_c = np.pad(y_predict_c, (n_days_sans_China, 0), 'constant') 
        y_predict_f = np.pad(y_predict_f, (n_days_sans_China, 0), 'constant')
        inflection_f = estimated_t0+n_days_sans_China
            
        extend_days_train = [i+len(x_train) for i in range(n_days_sans_China)]
        x_train     = np.append(x_train, extend_days_train)           
        y_train_c   = np.pad(y_train_c, (n_days_sans_China, 0), 'constant')
        y_train_f   = np.pad(y_train_f, (n_days_sans_China, 0), 'constant')
        inflection_c = estimated_t0_c+n_days_sans_China           
        
        ax0.plot(x_test, y_predict_c, linewidth=2, label='predict_'+country) 
        ax0.plot(x_train, y_train_c, linewidth=2, color='r', linestyle='dotted', label='train_'+country)
        ax0.set_title("Prediction vs Training for Confirmed Cases")
        ax0.set_xlabel("Number of days")
        ax0.set_ylabel("Confirmed Cases")
        ax0.legend()
        test_data.loc[test_data['Country_Region']==country,'ConfirmedCases'] = y_predict_c[-n_test_days:]
        
        ax1.plot(x_test, y_predict_f, linewidth=2, label='predict_'+country) 
        ax1.plot(x_train, y_train_f, linewidth=2, color='r', linestyle='dotted', label='train_'+country)    
        ax1.set_title("Prediction vs Training for Fatalities")
        ax1.set_xlabel("Number of days")
        ax1.set_ylabel("Fatalities")
        ax1.legend()
        test_data.loc[test_data['Country_Region']==country,'Fatalities'] = y_predict_f[-n_test_days:]             
    else: # use Province/State data when available
        isState = True
        state_list = []
        y_predict_c_dict = {}; y_train_c_dict = {}
        y_predict_f_dict = {}; y_train_f_dict = {}
        for state in df_country_train['Province_State'].unique():
            df_state_train = df_country_train[df_country_train['Province_State']==state] 
            df_state_test = df_country_test[df_country_test['Province_State']==state]   
            state_list.append(state)
            y_train_f = df_state_train['Fatalities']
            y_train_c = df_state_train['ConfirmedCases']   
            
            if y_train_f.empty== False:                 
                lower, upper = get_bounds_fatal (country, isState, y_train_f)
                popt_f, pcov_f = curve_fit(Gompertz, x_train, y_train_f, method='trf', bounds=(lower,upper))
                a_max, estimated_c, estimated_t0 = popt_f
                y_predict_f = Gompertz(a_max, estimated_c, x_test, estimated_t0) 
                y_predict_f_dict[state] =  y_predict_f
                y_train_f_dict[state]   =  y_train_f                
                #print('\nfatalities state ==>, max: ',a_max, ', slope: %.2f'% estimated_c, ', inflection point: ', 
                #    estimated_t0, ', r2 score: %.2f'% r2_score(y_train_f[:], y_predict_f[0:70]))                     
                                
            if y_train_c.empty == False:  
                lower_c, upper_c = get_bounds_confirm (country, isState, y_train_c)
                popt_c, pcov_c = curve_fit(Gompertz, x_train, y_train_c, method='trf', bounds=(lower_c,upper_c))
                a_max_c, estimated_c_c, estimated_t0_c = popt_c
                y_predict_c = Gompertz(a_max_c, estimated_c_c, x_test, estimated_t0_c)
                y_predict_c_dict[state] =  y_predict_c
                y_train_c_dict[state]   =  y_train_c
                #print('\nconfirmed state ==> max: ',a_max_c, ', slope: %.2f'% estimated_c_c, ', inflection point: ', 
                #  estimated_t0_c, ', r2 score: %.2f'% r2_score(y_train_c[:], y_predict_c[0:70]))
                            
        ## ====== Plot and Store the Results: ======
        ## ====== Move the x-axis of trained and test datasets to allign with dates in China ======       
        extend_days_test = [i+len(x_test) for i in range(n_days_sans_China)]
        x_test      = np.append(x_test, extend_days_test) 
        extend_days_train = [i+len(x_train) for i in range(n_days_sans_China)]
        x_train     = np.append(x_train, extend_days_train)           
            
        for state, y_predict in y_predict_f_dict.items():
            y_predict = np.pad(y_predict, (n_days_sans_China, 0), 'constant') 
            ax1.plot(x_test, y_predict, linewidth=2, label=state) 
            #ax1.legend(loc='center left',bbox_to_anchor=(1.0, 0.5)) 
            test_data.loc[(test_data['Country_Region']==country)&(test_data['Province_State']==state),'Fatalities'] = y_predict[-n_test_days:]
        for state, y_train in y_train_f_dict.items():
            y_train   = np.pad(y_train, (n_days_sans_China, 0), 'constant')
            ax1.plot(x_train, y_train, linewidth=2, color='r', linestyle='dotted', label='train_'+state)             
        ax1.set_title("Prediction vs Training for Fatalities")
        ax1.set_xlabel("Number of days")
        ax1.set_ylabel("Fatalities")   
        
        
        for state, y_predict in y_predict_c_dict.items():
            y_predict = np.pad(y_predict, (n_days_sans_China, 0), 'constant') 
            ax0.plot(x_test, y_predict, linewidth=2, label=state) 
            ax0.legend(loc='center left',bbox_to_anchor=(1.0, 0.5)) 
            test_data.loc[(test_data['Country_Region']==country)&(test_data['Province_State']==state),'ConfirmedCases'] = y_predict[-n_test_days:]
        for state, y_train in y_train_c_dict.items():
            y_train   = np.pad(y_train, (n_days_sans_China, 0), 'constant')
            ax0.plot(x_train, y_train, linewidth=2, color='r', linestyle='dotted', label='train_'+country+'_'+state)             
        ax0.set_title("Prediction vs Training for ConfirmedCases")
        ax0.set_xlabel("Number of days")
        ax0.set_ylabel("Confirmed Cases")   
        

In [ ]:
submit_data = pd.read_csv("../input/covid19-global-forecasting-week-4/submission.csv")#, index_col=0)

test_data['Fatalities'] = test_data['Fatalities'].fillna(0.0).astype(int)
test_data['ConfirmedCases'] = test_data['ConfirmedCases'].fillna(0.0).astype(int)

#submit_data['Country_Region'] = test_data['Country_Region']
#submit_data['Date'] = test_data['Date']
submit_data['Fatalities'] = test_data['Fatalities'].astype('int')
submit_data['ConfirmedCases'] = test_data['ConfirmedCases'].astype('int')

submit_data.to_csv('submission.csv', index=False)

#submit_data = submit_data[['Date','Country_Region','ConfirmedCases', 'Fatalities']]
submit_data.head()

In [ ]:
submit_data_1 = submit_data.groupby('Country_Region')[['ConfirmedCases', 'Fatalities']].sum().reset_index()
submit_data.to_csv('prova.csv', index=False)




In [ ]:
select_countries ["US"] = submit_data_1[(submit_data_1['Country_Region'] == 'US')]
select_countries

# **4. NLTK Sentiment Analysis** <a id="section4"></a>

In [ ]:
from IPython import display
import math
from pprint import pprint
import nltk
import seaborn as sns
sns.set(style='darkgrid', context='talk', palette='Dark2')

In [ ]:
BN = pd.read_csv("../input/twitter-covid19/BreakingNews.csv")
CNN = pd.read_csv("../input/twitter-covid19/cnn.csv")
NYT = pd.read_csv("../input/twitter-covid19/NYT.csv")

frames = [BN,CNN,NYT]
news = pd.concat(frames)

In [ ]:
import matplotlib.pyplot as plt

labels = news.username.unique()
sizes = news.username.value_counts()
explode = (0.1, 0, 0)
fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, autopct='%.1f%%', shadow=True, startangle=90)
ax.set_aspect('equal')
plt.show()


In [ ]:
text = news.text

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
results = []

for line in text:
    pol_score = sia.polarity_scores(line)
    pol_score['text'] = line
    results.append(pol_score)

pprint(results[:3], width=100)

In [ ]:
df = pd.DataFrame.from_records(results)
df.head()

In [ ]:
df['label'] = 0
df.loc[df['compound'] > 0.2, 'label'] = 1
df.loc[df['compound'] < -0.2, 'label'] = -1
df.head()

In [ ]:
df2 = df[['text', 'label']]
df.label.value_counts()

In [ ]:
print("Positive news:\n")
pprint(list(df[df['label'] == 1].text)[:5], width=200)

print("\nNegative news:\n")
pprint(list(df[df['label'] == -1].text)[:5], width=200)

In [ ]:
df.label.value_counts(normalize=True) * 100

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

counts = df.label.value_counts(normalize=True) * 100

sns.barplot(x=counts.index, y=counts, ax=ax)

ax.set_xticklabels(['Negative', 'Neutral', 'Positive'])
ax.set_ylabel("Percentage")

plt.show()

In [ ]:
import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
tokenizer = RegexpTokenizer(r'\w+')
stop_words = stopwords.words('english')

def process_text(headlines):
    tokens = []
    for line in headlines:
        line = line.lower()
        toks = tokenizer.tokenize(line)
        toks = [t for t in toks if t not in stop_words]
        tokens.extend(toks)
    
    return tokens

In [ ]:
pos_lines = list(df[df.label == 1].text)

pos_tokens = process_text(pos_lines)
pos_freq = nltk.FreqDist(pos_tokens)

pos_freq.most_common(20)

In [ ]:
from wordcloud import WordCloud,STOPWORDS
df_pos = df[df.label == 1].text
df_neg = df[df.label == -1].text

def wordcloud_draw(data, color = 'black'):
    words = ' '.join(data)
    cleaned_word = " ".join([word for word in words.split()
                            if 'http' not in word
                                and not word.startswith('@')
                                and not word.startswith('#')
                                and word != 'RT'
                            ])
    wordcloud = WordCloud(stopwords=STOPWORDS,
                      background_color=color,
                      width=2500,
                      height=2000
                     ).generate(cleaned_word)
    plt.figure(1,figsize=(13, 13))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()
    
print("Positive words")
wordcloud_draw(df_pos,'white')
print("Negative words")
wordcloud_draw(df_neg)